### This notebook generates transcipts using Google Speech-To-Text API. It saves the transcipt and apply standard preprocessing to the files. 

In [1]:
import io
import os
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
from smart_open import smart_open

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/storage/gluster/vol1/SHARED/HOMEFOLDERS/ceccarelli/Work/Bipolar/authen.json"
print(os.environ['HOME'])
print(os.environ.get('KEY_THAT_MIGHT_EXIST'))
os.environ

/home/ceccarelli
None


environ{'PATH': '/sbin:/usr/sbin:/bin:/usr/bin:/usr/local/bin',
        'LANG': 'en_US.UTF-8',
        'JUPYTERHUB_API_TOKEN': '3e0e1bb9840749a99912f046e40f12bf',
        'JPY_API_TOKEN': '3e0e1bb9840749a99912f046e40f12bf',
        'JUPYTERHUB_CLIENT_ID': 'jupyterhub-user-ceccarelli',
        'JUPYTERHUB_HOST': '',
        'JUPYTERHUB_OAUTH_CALLBACK_URL': '/user/ceccarelli/oauth_callback',
        'JUPYTERHUB_USER': 'ceccarelli',
        'JUPYTERHUB_SERVER_NAME': '',
        'JUPYTERHUB_API_URL': 'http://127.0.0.1:8081/hub/api',
        'JUPYTERHUB_ACTIVITY_URL': 'http://127.0.0.1:8081/hub/api/users/ceccarelli/activity',
        'JUPYTERHUB_BASE_URL': '/',
        'JUPYTERHUB_SERVICE_PREFIX': '/user/ceccarelli/',
        'USER': 'ceccarelli',
        'HOME': '/home/ceccarelli',
        'SHELL': '/bin/bash',
        'JPY_PARENT_PID': '11715',
        'TERM': 'xterm-color',
        'CLICOLOR': '1',
        'PAGER': 'cat',
        'GIT_PAGER': 'cat',
        'MPLBACKEND': 'module://ipyker

In [3]:
def get_sample(partition, index):
    """retrieve the sample name
    """
    # para partition: which partition, train/dev/test
    # para index: the index of sample
    if index < 0:
        print("\nINCORRECT INDEX INPUT")
        return 
    sample_name = ''
    if partition == 'train':
        if index > 104:
            print("\nSAMPLE NOT EXIST")
        else:
            sample_name = 'train_' + str(index).zfill(3)
    elif partition == 'dev':
        if index > 60:
            print("\nSAMPLE NOT EXIST")
        else:
            sample_name = 'dev_' + str(index).zfill(3)
    elif partition == 'test':
        if index > 54:
            print("\nSAMPLE NOT EXIST")
        else:
            sample_name = 'test_' + str(index).zfill(3)
    else:
        print("\nINCORRECT PARTITION INPUT")
    return sample_name

In [4]:
def load_audio_file(partition, index, gcs=False, verbose=False):
    """load audio files for transcribing
    """
    # para partition: which partition, train/dev/test
    # para index: the index of sample
    # para verbose: whether or not to output more results
    # return: array of audio filenames 
    audio_dir = '/home/ceccarelli/Work/Bipolar/recordings/recordings_audio' if not gcs else 'gs://mytranscripts'
    audio_list = []

    if not partition and not index:
        len_train = 104
        len_dev = 60
        len_test = 54

        for i in range(len_train):
            filename = get_sample('train', (i+1)) + '.wav'
            audio_list.append(os.path.join(audio_dir, filename))
            if verbose:
                print("load audio file:", audio_list[-1])

        for j in range(len_dev):
            filename = get_sample('dev', (j+1)) + '.wav'
            audio_list.append(os.path.join(audio_dir, filename))
            if verbose:
                print("load audio file:", audio_list[-1])

        for k in range(len_test):
            filename = get_sample('test', (k+1)) + '.wav'
            audio_list.append(os.path.join(audio_dir, filename))
            if verbose:
                print("load audio file:", audio_list[-1])

    elif partition and index:
        filename = get_sample(partition, index) + '.wav'
        audio_list.append(os.path.join(audio_dir, filename))
        if verbose:
            print("load audio file:", audio_list[-1])
    
    return audio_list

In [5]:
def save_transcript(partition, index, transcript):
    save_dir = '/home/ceccarelli/Work/Bipolar/transcripts'
    filename = get_sample(partition, index) + '.txt'
    with smart_open(os.path.join(save_dir, filename), 'w', encoding='utf-8') as output:
        output.write(transcript)
        output.write("\n")
    output.close()

In [6]:
class Speech2Text():
    def __init__(self, partition, index, gcs=False):
        self.client = speech.SpeechClient()
        self.filename = load_audio_file(partition, index, gcs=gcs, verbose=True)
        self.text = ""
        if len(self.filename) == 1:
            print("\ntranscribing audio file %s" % self.filename)
        if not gcs:
            self.transcribe()
        else:
            self.transcribe_gcs()
        if len(self.text) != 0:
            save_transcript(partition, index, self.text)

    def transcribe(self):
    
        with io.open(self.filename[0], 'rb') as audio_file:
            content = audio_file.read()
        audio = types.RecognitionAudio(content=content)
        config = types.RecognitionConfig(
            encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=16000,
            language_code='tr-TR'
        )

       
        operation = self.client.long_running_recognize(config, audio)
        

        print("\nwaiting for operation to complete ...")
        response = operation.result(timeout=200)

       
        for result in response.results:
            print(u"Transcript: {}".format(result.alternatives[0].transcript))
            print("Confidence: {}".format(result.alternatives[0].confidence))
            self.text += result.alternatives[0].transcript
       

    def transcribe_gcs(self):
        
        audio = types.RecognitionAudio(uri=self.filename[0])
        config = types.RecognitionConfig(
            encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=16000,
            language_code='tr-TR'
        )

        operation = self.client.long_running_recognize(config, audio)

        print("\nwaiting for operation to complete ...")
        response = operation.result(timeout=500)

        
        for result in response.results:
            print(u"Transcript: {}".format(result.alternatives[0].transcript))
            print("Confidence: {}".format(result.alternatives[0].confidence))
            self.text += result.alternatives[0].transcript
        

In [ ]:
for i in range(1, 105):
    Speech2Text('train', i, gcs=True)


load audio file: gs://mytranscripts/train_001.wav

transcribing audio file ['gs://mytranscripts/train_001.wav']

waiting for operation to complete ...
Transcript: Her sayfada izle gelecek görevinizi tamamlamak sona başlıklar başlat basınız
Confidence: 0.837432324886322
Transcript:  bu araya neler geldiğini anlar mısınız
Confidence: 0.8190560936927795
Transcript:  gelecek Onlar sen buradaki resmi anlar mısınız size ne düşündüğünü hissetti bu sapan bir insan
Confidence: 0.8142634630203247
Transcript:  bu hale gelmiş düşünüyor
Confidence: 0.8639723658561707
Transcript:  ama zor yani
Confidence: 0.47713714838027954
Transcript:  sizce
Confidence: 0.8037674427032471
Transcript:  kötü bir hatıranın anlar mısınız lütfen 30'a kadar sayar mısınız 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 29 30 1'den 30'a kadar hızla bir şekilde sayarmısınız 1 2 3 4 5 6 7 8 9 10 11 12 13 14 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 gelecek Onlar sayfadaki resmi alır mısınız size n

/home/ceccarelli/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



waiting for operation to complete ...
Transcript: yapa yapa Dükkanı eden bir Tüccar
Confidence: 0.6874954104423523
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26
Confidence: 0.9204601049423218
Transcript:  her zaman para Saadet getiriliyor
Confidence: 0.8602351546287537
Transcript:  mutluluk olsun gerek yok
Confidence: 0.6518509984016418
Transcript:  dilatör Ne aldın mı
Confidence: 0.6295502185821533
load audio file: gs://mytranscripts/train_003.wav

transcribing audio file ['gs://mytranscripts/train_003.wav']

waiting for operation to complete ...
Transcript: bu çalışmaya katılan kabul ettiğiniz için teşekkür ederim
Confidence: 0.8344996571540833
Transcript:  Lütfen her sayfada size verecek görevler tamamladıktan sonra Boşluk tuşuna basınız buraya neden geldiniz anlatır mısınız
Confidence: 0.8877691626548767
Transcript:  buralara gelecek olan sayfadaki resim anlatır mısınız siz neler düşünüyorsunuz düşün düşün yani buradaki amcaya ben eve de yapmış 


waiting for operation to complete ...
Transcript: duvar
Confidence: 0.6289302110671997
Transcript:  üzüntülü çok çok üzüldüm Çok üzüldüm
Confidence: 0.7756742238998413
Transcript:  19 20 21 22 23 24 25 26 28 29 20 22 23 24 25 26 27 tablosu var diyorlar
Confidence: 0.7558627128601074
Transcript:  gitmek istiyorum
Confidence: 0.7831482291221619
load audio file: gs://mytranscripts/train_007.wav

transcribing audio file ['gs://mytranscripts/train_007.wav']

waiting for operation to complete ...
Transcript: sıkıntı olduğu için
Confidence: 0.8017128109931946
Transcript:  düşünceleri
Confidence: 0.9167936444282532
Transcript:  adın ayrılma
Confidence: 0.6001500487327576
Transcript:  yani çocuklarından ayrı kalmak
Confidence: 0.8853437304496765
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 20
Confidence: 0.936693549156189
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.9204371571540833
Transcrip


waiting for operation to complete ...
Transcript: neden geldin bugünümü genelde bugün Bugün evden oranında danışma tarafından Eylül hocanın Teslime davet edildi ve zevkle geldim
Confidence: 0.7832848429679871
Transcript:  hüzünlü bir anda oturuyor sandalyede başında
Confidence: 0.7979904413223267
Transcript:  Çünkü bir hatırası anlatır mısın
Confidence: 0.704138994216919
Transcript:  Çok çok kötüydü Benim için bir asker yolculama ya gitmiştik
Confidence: 0.8872212767601013
Transcript:  eve misafir olduk açılmış çantam evde bir tanımadım Arkadaşlar bence çok kötü kararsız kalmıştım o
Confidence: 0.749280571937561
Transcript:  25 26 27 28 29 30 18 19 20 21 23 24 25 26 28 29 30
Confidence: 0.8003469109535217
Transcript:  kalabalık sıkıntı
Confidence: 0.9442601203918457
Transcript:  yatağın içinde çok hijyenik olmayan bir ortam
Confidence: 0.9442601203918457
Transcript:  Aha bir tane
Confidence: 0.6193028688430786
Transcript:  en güzel oyunlardan biri erkek arkadaşım çok güzel bir sürpriz

...


waiting for operation to complete ...
Transcript: buraya neden geldiniz anlatır mısınız 2 tane arkadaşım bir tane Arkadaşım vefat etti 2 gün uyumadım ilaçlarımı içmedim bilerek de ilaçlarımı içmek istemedim gata'daki Doktorlar kontrole gittim Yüzümü uykusuz gördükleri için beni buraya sevk ettiler onlardan şikayetçiyim şahsen benim hiçbir sıkıntım yoktu ben iki gün uyumadım gelecek olan sayfadaki neler düşünürdü hissettirdi zamanında çok hata yapmış yalnız kalmış ama sonuçta bir evi var Bu da büyük bir başarı alacak
Confidence: 0.9201155304908752
Transcript: kötü rahat ranza anlatır mısınız
Confidence: 0.6335986256599426
Transcript:  maalesef kötü bir hatıram aklıma gelmiyor bir de 30 e kadar sayar mısın 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.9029044508934021
Transcript:  hızlı bir şekilde sayarmısınız 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 gelecek olan sayfadaki resmi anlatır mısınız siz


waiting for operation to complete ...
Transcript: çalışmaya katılmayı kabul ettiğin için teşekkür ederiz Size verecek Ondan sonra Boşluk tuşuna basınız buraya neden geldiğinizde anlatır mısınız çok sevindim kendimi kontrol edemiyorum yüksekti o yüzden geldim gelecek olan sayfadaki resim anlatır mısınız size neler düşündürüyor ve Nefret ettirdi
Confidence: 0.8901668787002563
Transcript:  yine aynı adam ateşin Başında Çaresizce kapaklı 61 yaşlı amca
Confidence: 0.7032896876335144
Transcript:  yüzünü kapatmış
Confidence: 0.9442601203918457
Transcript:  kötü bir Hatırınızı anlatır mısınız kötü bir hatıram
Confidence: 0.870604395866394
Transcript:  kötü bir hatam olmadı ama
Confidence: 0.8056557178497314
Transcript:  hatıram anlatır mısınız
Confidence: 0.6209567189216614
Transcript:  hatırlamıyorum
Confidence: 0.9442601203918457
Transcript:  kötü bir hafta
Confidence: 0.717942476272583
Transcript:  kötü bir hatıra herhalde annemin annemin bana şey yapmasını başarısız olduğumda hazırlanması


waiting for operation to complete ...
Transcript: teşekkür ederim
Confidence: 0.9442601203918457
Transcript:  kabul ettiğin için teşekkür ederim Ben de teşekkür ederim Anlayışınız için çok hızlı konuşuyorum Çok çabuk konuşuyordum kendini susturamıyorum beyni Melike demiyorum artık girecek olan sayfadaki resmi anlatır mısınız Eğer Düşündürdü beni düşünen bir adam yıkılmış bir adam sobanın başında kar olan bir adam sanki sanki böyle güzel insan yüzünü kapatmış yüzünü kapatmış bu Tamam utangaç insan kötü bir hastalığınız anlatır mısınız kötü bir alacağım
Confidence: 0.8891438841819763
Transcript:  futbol oynamak istiyordum çocukken Amatör olarak yazılmak istiyorum Bir türlü yazdım Ama babam izin verdi en kötü hatıra ama 1'den 30'a kadar sayar mısınız 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 1 2 3 4 5 6 7 8 9 10 11 12 13 hissettirdi mutlu bir aile çocuklar ile beraber çekirdek ve aile çocuklarla beraber uyuyorlar Bir de tavuk var ortada tavuk Bir de


waiting for operation to complete ...
Transcript: tedavi üzerine geldim uyguluyorum yatıyorum ben bir daha yapmamak üzere Bundan sonra da hiç bir aksilik daha bir şey olmayacak Ben kendimi iyi hissediyorum hocam
Confidence: 0.8803765177726746
Transcript: baktığımızda çok üzüyor çok dertli çok üzüntülü ve kederli gibi bir ifade
Confidence: 0.8188291788101196
Transcript:  Aslında yok Ne olacak ki istemiyorum Çünkü hocam
Confidence: 0.817946195602417
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 22 23 24 25 26 27 28 29 30
Confidence: 0.937120795249939
Transcript:  5 6 7 8 9 10 11 12 13 14 15 23 24 25 26 27 28 29 30
Confidence: 0.9357835054397583
Transcript:  dördümüz resimdeki şeyler var
Confidence: 0.5480197072029114
Transcript:  beni çok kötü oluyor bunun cevaplaması olur mu hocam cevap verin olduğum zaman işinden çok başarılı olduğum zaman çok planlı olalım oldu annen güzel yazdın yapmamı yaptığım zaman onu çok huzurlu bir aile annemi de çok seviyorum onları Almazsan


waiting for operation to complete ...


In [7]:
for i in range(1, 61):
    Speech2Text('dev', i, gcs=True)

load audio file: gs://mytranscripts/dev_001.wav

transcribing audio file ['gs://mytranscripts/dev_001.wav']

waiting for operation to complete ...
Transcript: Bozkurt
Confidence: 0.7186861038208008
Transcript:  bir adam var ağlıyor gibi oraya gelmektir yani bana göre 1'den 30'a kadar sayarmış 1 2 3 4 5 6 7 8 9 10 11 12 13 14 20 21 22 23 24 25 26 27 28 29 30 32 34 5 6 7 8 9 10 11 12 13 14 15 16 17 18 26 27 28 29 30
Confidence: 0.9014163613319397
load audio file: gs://mytranscripts/dev_002.wav

transcribing audio file ['gs://mytranscripts/dev_002.wav']


/home/ceccarelli/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



waiting for operation to complete ...
Transcript: daha sonra Boşluk tuşuna bas buraya neden Geldiniz diye anlatır mısınız bir bilgim yoktu
Confidence: 0.7366990447044373
Transcript:  arkadaşım kahve içmeye çıkardı Beni bundan sonra kahve içmeye çıkarken pasta yeriz dedi Direk buraya getirdi buraya geldi bu bilmiyorum ama sordum Neden buraya beni dedim senin iyilik Senin evin için getirdim dedi o kadar Başka yok
Confidence: 0.911749005317688
Transcript:  sayfadaki resmi Hanım anlatır mısınız size neler düşündürüyor
Confidence: 0.9000701904296875
Transcript:  ağlayan ya da bir şey düşünüyorum Başka bir şey
Confidence: 0.9224954843521118
Transcript:  en kötü kadro
Confidence: 0.7706199884414673
Transcript:  hastaneye gideceğiz Sen ettim Gerçekten Çünkü gerçekten ben güvendim Ama geri beni kasavet çıktı buraya kötü olur benim için 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 a kadar hızlı bir şekilde sayar mısınız 1-2-3-4-5-6-7-8-9-10 9 10 11 12 13 14 1


waiting for operation to complete ...
Transcript: Merhaba için geldim
Confidence: 0.8280112743377686
Transcript:  Tamam mutsuz karamsar kötü kötü anlama unuttum onu adam olmuş da o kadar
Confidence: 0.8094045519828796
Transcript:  mutluluk gel
Confidence: 0.7939786911010742
Transcript:  ekmek bitti
Confidence: 0.7829424738883972
load audio file: gs://mytranscripts/dev_011.wav

transcribing audio file ['gs://mytranscripts/dev_011.wav']

waiting for operation to complete ...
Transcript: neden gelin Bilemiyorum ki buraya
Confidence: 0.8294153809547424
Transcript:  bir sebep yok gibi hiçbir sıkıntı olmadan dolayı geldi mi eniştem mi onu getirdim yani
Confidence: 0.8040447235107422
Transcript:  örüşürüz İyi bir şey düşünmedim
Confidence: 0.7045319676399231
Transcript:  biraz hissediyorum ama biraz da az
Confidence: 0.6850574612617493
Transcript:  kötü bir hatamız anlatır mısın kötü bir yatağın insanlara karşı Gedik Yatırım insanlardan her zaman kalkıyor
Confidence: 0.8713017702102661
Trans


waiting for operation to complete ...
Transcript: hastalık nedeniyle gördüm iyileşmek için
Confidence: 0.7225103378295898
Transcript:  olmadı
Confidence: 0.8044869303703308
Transcript:  3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.8563975095748901
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.944259762763977
Transcript:  ailece çocuk
Confidence: 0.6441676020622253
Transcript:  horoz
Confidence: 0.8366574645042419
Transcript:  köpek Hepsi Bir yatakta bir yerde bir odada yatıyor
Confidence: 0.9442600011825562
Transcript:  evde çalışacağım da beraber olmak
Confidence: 0.5542452931404114
load audio file: gs://mytranscripts/dev_017.wav

transcribing audio file ['gs://mytranscripts/dev_017.wav']

waiting for operation to complete ...
Transcript: Rica ederim
Confidence: 0.5949957370758057
Transcript:  Neden geldin oturuyorsunuz
Confidence: 0.5758349895477295
Transcript:  peşini olan ba

...


waiting for operation to complete ...
Transcript: apartmana geldim nasıl gittiğini görmek için kontrol amaçlı geldim
Confidence: 0.7916516065597534
Transcript:  bir adam var
Confidence: 0.5664114356040955
Transcript:  Pek bir şey göremiyorum yani Ateş olmayan insanlardan kendini saklamaya çalışıyor zorladığımda Depresyonda gibi sanki ya da evi yanıyor öyle bir şey de olabilir Bilmiyorum belki de öyle bir şeyler
Confidence: 0.7724748849868774
Transcript:  gel de şeyler var ama
Confidence: 0.7687387466430664
Transcript:  bilmiyorum çözemedim kötü bir hatırını damlatılırsa sonra annemle yaşadığım şey Annem de işte tartıştık ondan sonra babam gelince hatıram oldu bayağı ağır sözler söyledim ona çok Sonra çok pişman oldum ama iş işten geçti yani 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.9224101901054382
Transcript:  Bu da mutlu bir aile tablosu
Confidence: 0.7


waiting for operation to complete ...
Transcript: sayfada diye yazıyor geçmedi
Confidence: 0.8602256774902344
Transcript:  neden geldiğinizde anlatır mısınız
Confidence: 0.6684274673461914
Transcript:  yazacak mıyım ben karakola çağırdı imza atarsın diye
Confidence: 0.758845865726471
Transcript:  karikatür
Confidence: 0.6414871215820312
Transcript:  anlatımı
Confidence: 0.50618577003479
Transcript:  o kadar sayarmış 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 23 24 25 26 27 28
Confidence: 0.8911570906639099
Transcript:  3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.9442278146743774
Transcript:  Maşallah hepsini
Confidence: 0.7427501082420349
Transcript:  yatakta yatıyorlar
Confidence: 0.7226803302764893
Transcript:  en güzel adına ne zaman çalışırsın en güzel hatırlamam
Confidence: 0.654495120048523
load audio file: gs://mytranscripts/dev_040.wav

transcribing audio file ['gs://mytranscripts/dev_040.wav']

waiting for operation to


waiting for operation to complete ...
Transcript: katılmayan
Confidence: 0.934402585029602
Transcript:  dışına
Confidence: 0.4132138192653656
Transcript:  Fethiye anlatır mısın düşünürdü
Confidence: 0.713408350944519
Transcript:  memeni durmadan kadar 1 2 3 4 5 6 7 8 9 10 11 12 13 21 22 23 24 25 26 27 28 29 30 30 a kadar hızlı 1-2-3-4-5-6-7-8-9-10 neler düşündürüyor
Confidence: 0.8951117396354675
Transcript:  tavuklarda almış kürenin tanımı
Confidence: 0.6643380522727966
Transcript:  6 tane çocuğuyla köpek kedi köpek köpek ne yapıyor ne ediyor
Confidence: 0.7395645976066589
Transcript:  teşekkür ederiz
Confidence: 0.8470234274864197
load audio file: gs://mytranscripts/dev_047.wav

transcribing audio file ['gs://mytranscripts/dev_047.wav']

waiting for operation to complete ...
Transcript: blades amaçlı kontrol amaçlı yardım
Confidence: 0.7544853091239929
Transcript:  Çok düşünceli burada
Confidence: 0.83885657787323
Transcript:  kendini üzüntülü hissediyor
Confidence: 0.88614809513092


waiting for operation to complete ...
Transcript: hastayım
Confidence: 0.7744623422622681
Transcript:  bilmiyorum ne olduğunu
Confidence: 0.8121981620788574
Transcript:  adam var yani
Confidence: 0.7031136751174927
Transcript:  19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.9414106607437134
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29
Confidence: 0.9371141195297241
Transcript:  aile resmi
Confidence: 0.8061172366142273
Transcript:  Nasıl yapalım
Confidence: 0.9408972859382629
load audio file: gs://mytranscripts/dev_053.wav

transcribing audio file ['gs://mytranscripts/dev_053.wav']

waiting for operation to complete ...
Transcript: Neyi neden geldiniz anlatımsız çağırdınız için geldim Test yapmak için kendin tedavi
Confidence: 0.8222938776016235
Transcript:  bu adam şey düzgün bir adam bir şey bir şeyden kendini korumaya çalışıyor galiba kendi yüzünü kapatmış korkutucu bir olay yaşamış olabilir Ondan dolayı anlatır mısınız şu an aklıma 


waiting for operation to complete ...
Transcript: oraya
Confidence: 0.7989949584007263
Transcript:  duygu durum bozukluğu teşhisi kondu için
Confidence: 0.8058683276176453
Transcript:  düzeltmesini düzeltilmesi için yarın aşağı yukarı hayatın normale dönmesi için
Confidence: 0.6347830891609192
Transcript:  kızlar için noktalama
Confidence: 0.8866294622421265
Transcript:  her şeyini yitirmiş bir 7 haftada
Confidence: 0.5057300925254822
Transcript:  çocuğu yok mu tek başına kalmış
Confidence: 0.9127988219261169
Transcript:  adres yaptığı zamanında hataların cezasını çekti acaba
Confidence: 0.8281504511833191
Transcript:  kültür arttıran
Confidence: 0.7447852492332458
Transcript:  çok şey yaşadım hayatımda
Confidence: 0.9186592102050781
Transcript:  en son yaşadığım olayı anlatayım kavga etmiştim
Confidence: 0.8387160301208496
Transcript:  benimsin otocam hayatım anca
Confidence: 0.646037220954895
Transcript:  alanlardan bir tanesidir bir nokta kadar sayar mısınız
Confidence: 0.845933556

In [7]:
for i in range(1, 55):
    Speech2Text('test', i, gcs=True)

load audio file: gs://mytranscripts/test_001.wav

transcribing audio file ['gs://mytranscripts/test_001.wav']

waiting for operation to complete ...
Transcript: Teşekkür ederim başkanım ne yapıyoruz
Confidence: 0.7814576625823975
Transcript:  tamam
Confidence: 0.9442601203918457
Transcript:  buraya neden gelmiştim yine böyle sinir bozukluğu Oldu kardeşim ve eşimle geldim
Confidence: 0.9442599415779114
Transcript:  öyle Şu an anlatabileceğim bu kafamda pek bir şey yok açıkça söylemek gerekirse
Confidence: 0.7461506128311157
Transcript:  gelecek olan Yinemi sensin Ben
Confidence: 0.6534532308578491
Transcript:  Bu tamamen
Confidence: 0.8118532299995422
Transcript:  duygusu karışmış kafası karışmış ne yapacağını bilmeyen bir adam hatırlattı bana
Confidence: 0.9253204464912415
Transcript:  evlendiğim ilk gündü ondan öncesi de vardı Elvan Hanım genç kız genç kızlık raporu almışlardı çok üzülmüştüm temiz olduğum halde
Confidence: 0.923585057258606
Transcript:  en kötü günüm oynasın
Confidenc

/home/ceccarelli/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL



waiting for operation to complete ...
Transcript: psikolojikmen bir çöküntü mu vardı buraya gelmezdi neler sağ olsun doktorlarımız Yardımcı oldular Demek ki Her insanın ihtiyacı var böyle bir ihtiyacın olan trafiği bana sağolsunlar sağladılar şükürler olsun yavaş yavaş iyi olduğunu hissediyorum
Confidence: 0.9130858778953552
Transcript: karamsarlığa bağlanmış
Confidence: 0.8117413520812988
Transcript:  Yarın biz Ama hayatta zarar etti ama ticarette zarar etti ve pişmanlık duygusu içinde
Confidence: 0.8525169491767883
Transcript:  kendisini kaybetmiş vaziyete gelmişti abimiz
Confidence: 0.8299591541290283
Transcript:  en kötü katran 2011 senesinde Almanya gitmek üzere bir yolculuk yaptım Yolculuk esnasında beni Nehire Tunar evde bıraktılar
Confidence: 0.8345626592636108
Transcript:  yanımda bir Suriyeli bayan olmak kaybeden 2 kişilik bir 15 saat kadar bir diyebilirim ona su boyumuzu başlayacak derecede 15 saatin sonunda gene de telefona ulaşılamıyor duk oradaki Romen balıkçılardan yard


waiting for operation to complete ...
Transcript: buraya neden geldiniz anlatır mısın diyor
Confidence: 0.8909186124801636
Transcript:  buraya neden geldim Bir kere kendimi Şikayet Edeyim bir yeri bir memlekette bir Müslümana eziyet edilirse O ülkeye Bela yaz 99 depremi benim yüzümden oldu kısa keseceğim kız o zaman
Confidence: 0.8773173689842224
Transcript:  Baba ölmüş Zalim koca Zalim işlerini abla
Confidence: 0.9133340120315552
Transcript:  Zalim evlatlar ve burada geçirdim ziyaretimize gelmezler onları göreyim uçağın abla ilaç içirdi beneday ilaç ilişki robot gibi oluyor bana nedir kastınız yayınlandı mı diye 2008 tarihinde
Confidence: 0.8929153680801392
Transcript:  Tayyip Erdoğan Beni Yerden Yere vurdu Peygamber Efendimizin diyeceksiniz ki bunu niye yalan söyledin Osmaneli Ne anlatayım ki bu kadar Sadece geçerli tek bir şey söyledi mi
Confidence: 0.8534997701644897
Transcript:  Osman hocanın telefon numarasını vermiş 0212 454 56 39 bastırmanın
Confidence: 0.7302268743515015
Tran


waiting for operation to complete ...
Transcript: bu çalışmaya katılmayı kabul ettiğiniz için teşekkür ederim Ben de teşekkür ederim Lütfen her sayfada size verilecek görevleri tamamladıktan sonra Boşluk Boşluk tuşuna basınız Suriye neden geldiniz anlatır mısınız manik atak geçirdim için geldi gelecek olan sayfadaki resmi anlatır mısınız size neler düşündürüyor ve hissettirdi üzüntülü bir anca tek kötü bir hal hatırımızı anlatır mısınız yok 1'den 30'a kadar sayar mısınız 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 1'den 30'a kadar hızlı bir şekilde sayarmısınız 1234567 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 sayfadaki resmi anlatır mısınız size neler düşündüğünü ve hissettirdi
Confidence: 0.9259907603263855
Transcript: mutlu aile tablosu yaşamımızdaki en güzel hatırımızı anlatır mısınız oğlumun doğuşu Teşekkürler Ben teşekkür ederim bitti
Confidence: 0.8617509007453918
load audio file: gs://mytranscripts/test_011.wav

transcribing audio 

...


waiting for operation to complete ...
Transcript: erken gittin mi
Confidence: 0.7273952960968018
Transcript: 1234567
Confidence: 0.8102849721908569
Transcript: 4 25 26 27 28 29 30
Confidence: 0.8432206511497498
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 28 29 30
Confidence: 0.944259762763977
Transcript:  Serhat durum ilk buraya o zaman güzel oldu
Confidence: 0.6733718514442444
Transcript:  arkadaş hırsız girmişti gördüm ondan korktum
Confidence: 0.8071386218070984
load audio file: gs://mytranscripts/test_045.wav

transcribing audio file ['gs://mytranscripts/test_045.wav']

waiting for operation to complete ...
Transcript: bir atak geçirdim o atak sonucunda Ailem beni buraya getirdi hatırlıyorum Şimdi de yanlış hatırlıyorum her şeyi Anneme babama kardeşlerime tanıyamıyorum başkaları imiş gibi geliyordu şimdi hatırladım Bir ara geçirdiğim için geldim Bir uykusuzluk dönüverdi yemek yememe dönemi vardı o yüzden getirdiler hastaneye
Confidence: 0.8862


waiting for operation to complete ...
Transcript: buraya neden geldiğinizde anlatır mısınız
Confidence: 0.8605930805206299
Transcript:  buraya neden geldiğinizde anlatır mısınız
Confidence: 0.763820469379425
Transcript:  cümleler toparlıyorum
Confidence: 0.9442601203918457
Transcript:  eşime böbrek sancım var dedim var dedim Dinlemedi için
Confidence: 0.8528318405151367
Transcript:  ve senin psikolojik sorunların var başka böbrek taşı falan sende yok dediği için beni hastane hastane tuttular Kolumdan gezdirdiler en son yine geldi ama şu anda rahatım hiçbir şeyde
Confidence: 0.8827152252197266
Transcript:  hiçbir şey derken bir çocuğum var aklımda çünkü o kadar şeyi o kadar yanlışları benim düzeltmeyi daha takatim yok yani
Confidence: 0.8685505986213684
Transcript:  düzelmesi benim elimden gelen bu kadar ya bu can çıktığı zaman asıl
Confidence: 0.8408795595169067
Transcript:  asıl hesap verecek olduğumuz gün gelir o gün elbette haklı haksıza birbirleri ile karşılaşıldı
Confidence: 0.81


waiting for operation to complete ...
Transcript: geçtin mi
Confidence: 0.924527645111084
Transcript:  Onu biliyoruz Boşluk tuşuna basıyor
Confidence: 0.6909281611442566
Transcript:  Ne yapayım Benim yaşadıklarımı kaldı kafamda biriktirdim biriktirdim hiç kimse suçlu Suçlu benim her hayırda bir şer Vardır her şeyde bir hayır vardır
Confidence: 0.8713194727897644
Transcript:  Sadece şimdi diyorum ki Allah Allah cezanı verdi akılsız iki tane doktora Eline Düştüm Bir tanesi Asistan bir tanesi üst doktor
Confidence: 0.7939516305923462
Transcript:  duanın kabul oluyor mu oluyor mu oluyor mu sürekli üstüme geliyor söylüyorum kabul oluyor
Confidence: 0.7971886992454529
Transcript:  Allah'la konuşuyorum sonra konuşuruz sürekli onu söylüyor Allahu Teala ile Musa aleyhisselam görünmeden konuş
Confidence: 0.7935282588005066
Transcript:  Evet Allah Celle Celalühü konuşmak kur'an-ı Kerim okumak konuşmayı yapacağız
Confidence: 0.7490711212158203
Transcript:  geldiğimizden evladım çok acil
Confidenc


waiting for operation to complete ...
Transcript: gelmek istedim
Confidence: 0.6743865013122559
Transcript:  anlatmak için geldi
Confidence: 0.7633171677589417
Transcript:  kızımla ilgili bir olay
Confidence: 0.9431812167167664
Transcript:  düşünceli bir resim
Confidence: 0.8164893388748169
Transcript:  o kadar çok varken devam et
Confidence: 0.6265614628791809
Transcript:  Yürümek istiyorum
Confidence: 0.7425501942634583
Transcript:  1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.9216872453689575
Transcript:  3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30
Confidence: 0.9282010197639465
Transcript:  ay
Confidence: 0.8249027132987976
Transcript:  çok tatlı uyuyor hepsi bir arada anne baba çocuklar Kardeşler
Confidence: 0.8319090008735657
Transcript:  tavuk bile yapmadım ben
Confidence: 0.591917872428894
Transcript:  çocukların kucağına aldı mı
Confidence: 0.5943695902824402
load audio file: gs://mytranscripts/te


waiting for operation to complete ...
Transcript: Rica ederim Lütfen her sayfada Side verilecek görevliye tamamladıktan sonra Boşluk tuşuna basılı buraya neden geldiğiniz anlatır mısınız iki uçlu duygu durum bozukluğu bipolar teşhisi hastalığı yüzünden geldi ve tedavisini oluyorum
Confidence: 0.877714216709137
Transcript:  tedavisine oluyorum
Confidence: 0.6663026213645935
Transcript:  yani çok neşeli çok sinirli çok affettim olmam yüzünden geldi gelecek olan sayı özetini anlatır size neler düşündürüyor hissettirdi
Confidence: 0.8154633641242981
Transcript:  yalnız başına düşünüp sinir
Confidence: 0.9442601203918457
Transcript:  krizi gibi
Confidence: 0.8631167411804199
Transcript:  sinir krizi geçiren bir insanı bir adamı anlatıyor yani üzüntülü işte sinirleri bozuk bir insanı ağlatacak
Confidence: 0.8378902077674866
Transcript:  kötü bir hatıra ağınızda anlatır mısınız
Confidence: 0.8403654098510742
Transcript:  kötü bir hatıra
Confidence: 0.7902306318283081
Transcript:  kötü bir ha

In [ ]:
def preproc_transcript(partition):
    if partition == 'all':
        len_train = 104
        len_dev = 60
        len_test = 54
        in_path = '/home/ceccarelli/Work/Bipolar/transcripts'
        docs = '/home/ceccarelli/Work/Bipolar/transcripts_processed/All_transcripts.txt'

        with smart_open(docs, 'w', encoding='utf-8') as output:
            for i in range(len_train):
                filename = get_sample('train', (i+1)) + '.txt'
                with smart_open(os.path.join(in_path, filename), 'r', encoding='utf-8') as input:
                    for line in input:
                        output.write(line)
                input.close()
            for j in range(len_dev):
                filename = get_sample('dev', (j+1)) + '.txt'
                with smart_open(os.path.join(in_path, filename), 'r', encoding='utf-8') as input:
                    for line in input:
                        output.write(line)
                input.close()
            for k in range(len_test):
                filename = get_sample('test', (k+1)) + '.txt'
                with smart_open(os.path.join(in_path, filename), 'r', encoding='utf-8') as input:
                    for line in input:
                        output.write(line)
                input.close()
        output.close()

    else:
        
        lengths = [104, 60, 54]
        partitions = ['train', 'dev', 'test']
        filenames = ['Train_transcripts.txt', 'Dev_transcripts.txt', 'Test_transcripts.txt']
        outpath = '/home/ceccarelli/Work/Bipolar/transcripts_processed/'
        in_path = '/home/ceccarelli/Work/Bipolar/transcripts'
        
        for i in range(len(lengths)):
            docs = os.path.join(outpath, filenames[i])
            with smart_open(docs, 'w', encoding='utf-8') as output:
                for j in range(lengths[i]):
                    filename = get_sample(partitions[i], (j+1)) + '.txt'
                    print("%s read" % filename)
                    with smart_open(os.path.join(in_path, filename), 'r', encoding='utf-8') as input:
                        for line in input:
                            output.write(line)
                    input.close()
            output.close()

In [ ]:
preproc_transcript('no')